In [1]:
import numpy as np
import pandas as pd
import nltk
import string
import re
import matplotlib.pyplot as plt
import unicodedata
import sys
import collections
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
stopwords = nltk.corpus.stopwords.words('english')
unicodePuncs = [chr(i) for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P')]
stopwords += ([i for i in string.punctuation] + ['--', '\'\'', '``', '...'])
stopwords += unicodePuncs

In [3]:
class BOWExtractor():      
    def __init__(self, ngrams_to_take = {1: 100, 2: 50}, stopwords=[], tokenizer = nltk.word_tokenize, unicodeNormalize = True, binary = False):
        self.ngrams_to_take = ngrams_to_take
        self.stopwords = stopwords
        self.tokenizer = tokenizer
        self.unicodeNormalize = unicodeNormalize
        self.binary = binary
    
    def hasnum(self,s):
        return any(i.isdigit() for i in s)
    
    def hasPunc(self, s):
        return any(i in string.punctuation for i in s)
    
    def preprocess(self, series):
        if self.unicodeNormalize:
            series = series.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii','ignore').decode('utf-8'))
        toks = series.apply(lambda x: list(filter(lambda z: z not in self.stopwords, map(lambda y: y.lower(),self.tokenizer(x)))))
        toks = toks.apply(lambda x: [i for i in x if not self.hasPunc(i) and not self.hasnum(i)])
        return toks
    
    def getngrams(self, lst, n=2):
        ans = []
        for i in range(len(lst) - n+1):
            ans.append([lst[i+j] for j in range(n)])
        return ans
    
    def flatten(self, x):
        if isinstance(x, (np.ndarray, list, tuple)):
            lst = []
            for i in x:
                lst += self.flatten(i)
            return lst
        else:
            return [x]
    
    def fit(self, series): #input is a pandas series of strings
        assert(isinstance(series.iloc[0], str))
        toks = self.preprocess(series)
        self.most_common_ngrams = {}
        for i in self.ngrams_to_take.keys():
            if i == 1:
                self.most_common_ngrams[i] = {i[0]: i[1] for i in nltk.FreqDist(self.flatten(toks.values)).most_common(self.ngrams_to_take[i])}
            else:
                ngrams = [j for i in toks for j in self.getngrams(i)]
                self.most_common_ngrams[i] = {i[0]: i[1] for i in collections.Counter(map(tuple, ngrams)).most_common(self.ngrams_to_take[i])}
        self.features = [j for i in self.most_common_ngrams.keys() for j in self.most_common_ngrams[i].keys()]
        return self
            
    def transform(self, series): #input is a pandas series of strings, output is a 2d numpy array, in the order of self.features
        ngrams = {}
        toks = self.preprocess(series)
        for i in self.ngrams_to_take.keys():
            if i == 1:
                ngrams[1] = toks
            else:
                ngrams[i] = toks.apply(lambda x: self.getngrams(x, i))
        ans = np.zeros(shape = (len(series), len(self.features),), dtype='int16')
        # loop through each feature
        for count, i in enumerate(self.features):
            if isinstance(i, str):
                ans[:, count] = ngrams[1].apply(lambda x: x.count(i)).values
            else: #tuple
                ans[:, count] = ngrams[len(i)].apply(lambda x: x.count(i)).values
        if self.binary:
            ans = (ans>0).astype(int)
        return ans

#### Load Data

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [5]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
output_names = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [7]:
train[output_names].apply(pd.value_counts)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,144277,157976,151122,159093,151694,158166
1,15294,1595,8449,478,7877,1405


In [8]:
#traindf, valdf = train_test_split(train, test_size = 0.1, random_state = 42, stratify = train.threat)

#### Feature Extraction

In [9]:
bow= BOWExtractor(ngrams_to_take = {1:2400, 2:1600, 3:400},stopwords = stopwords).fit(train['comment_text'])

In [10]:
train_mat = bow.transform(train['comment_text']).astype('int16')

#### Model

In [12]:
final_models = []
gs_objs = []
for i in output_names:
    lr = LogisticRegression(random_state = 10)
    param_grid = {'C': 10**np.linspace(-2,2,5),
        'penalty': ['l2']
    }
    gs = GridSearchCV(estimator = lr, param_grid = param_grid, scoring = 'roc_auc', 
                      n_jobs = 2, cv = 3, refit = True, verbose = 10, return_train_score = True).fit(train_mat, train[i])
    final_models.append(gs.best_estimator_)
    gs_objs.append(gs)
    print('Completed calculation for', i)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] C=0.01, penalty=l2 ..............................................
[CV] C=0.01, penalty=l2 ..............................................
[CV] ..... C=0.01, penalty=l2, score=0.9351442743376353, total=  10.6s
[CV] C=0.01, penalty=l2 ..............................................


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   15.5s


[CV] ..... C=0.01, penalty=l2, score=0.9283337526834519, total=  13.2s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ..... C=0.01, penalty=l2, score=0.9279195451917107, total=  11.1s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ....... C=0.1, penalty=l2, score=0.938509175007358, total=  13.0s
[CV] C=0.1, penalty=l2 ...............................................


[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   35.4s


[CV] ........ C=0.1, penalty=l2, score=0.93414299720488, total=  11.9s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.9343704671806133, total=  12.5s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=1.0, penalty=l2, score=0.9357168132062554, total=  13.8s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=1.0, penalty=l2, score=0.9346370972570652, total=  11.9s
[CV] C=10.0, penalty=l2 ..............................................
[CV] ...... C=1.0, penalty=l2, score=0.9343512440210793, total=  13.2s
[CV] C=10.0, penalty=l2 ..............................................


...

[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.9154300155921871, total=  16.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.8666108505928694, total=  14.5s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=1.0, penalty=l2, score=0.8958248151849961, total=  11.8s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=1.0, penalty=l2, score=0.9154217882737287, total=  12.3s
[CV] C=10.0, penalty=l2 ..............................................
[CV] ...... C=1.0, penalty=l2, score=0.8613375447476961, total=  12.3s
[CV] C=10.0, penalty=l2 ..............................................


[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  1.3min


[CV] ...... C=10.0, penalty=l2, score=0.895865500085778, total=  13.0s
[CV] C=10.0, penalty=l2 ..............................................
[CV] ..... C=10.0, penalty=l2, score=0.9154137230699199, total=  13.1s
[CV] C=100.0, penalty=l2 .............................................
[CV] ..... C=10.0, penalty=l2, score=0.8626577659239547, total=  13.0s
[CV] C=100.0, penalty=l2 .............................................
[CV] .... C=100.0, penalty=l2, score=0.8958683310430888, total=  12.3s
[CV] C=100.0, penalty=l2 .............................................
[CV] ..... C=100.0, penalty=l2, score=0.915461871120799, total=  11.9s
[CV] .... C=100.0, penalty=l2, score=0.8614776928621245, total=   9.7s


[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  2.0min finished


Completed calculation for identity_hate


In [13]:
for a,b in zip(output_names, gs_objs):
    print(a,b.best_score_)

toxic 0.935674230897
severe_toxic 0.952322399807
obscene 0.953171741693
threat 0.925945811499
insult 0.929480784076
identity_hate 0.89180794951


#### Submission

In [14]:
test_mat = bow.transform(test['comment_text'])

In [18]:
pred = [model.predict_proba(test_mat)[:,1] for model in final_models]

In [19]:
for count,i in enumerate(output_names):
    test[i] = pred[count].flatten()

In [20]:
test[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].to_csv('data/answers/linear_baseline1.csv', index = False)